# Exploración de Datos - Feedback de Clientes

Análisis exploratorio del dataset de feedback de clientes.

In [17]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
sns.set_style('whitegrid')

In [18]:
# Cargar el dataset
df = pd.read_csv('datasets/feedback_clientes_v2.csv')
print(f"Dataset cargado: {df.shape[0]} filas y {df.shape[1]} columnas")

Dataset cargado: 4500 filas y 9 columnas


In [19]:
# Mostrar las columnas del dataset
print("Columnas del dataset:")
print("-" * 50)
for i, col in enumerate(df.columns, 1):
    print(f"{i}. {col}")
    
print("\n" + "=" * 50)
print(f"Total de columnas: {len(df.columns)}")

Columnas del dataset:
--------------------------------------------------
1. Feedback_ID
2. Transaccion_ID
3. Rating_Producto
4. Rating_Logistica
5. Comentario_Texto
6. Recomienda_Marca
7. Ticket_Soporte_Abierto
8. Edad_Cliente
9. Satisfaccion_NPS

Total de columnas: 9


In [20]:
# Análisis de valores nulos por columna
print("Valores Nulos por Columna:")
print("=" * 60)

nulos = df.isnull().sum()
porcentaje_nulos = (df.isnull().sum() / len(df)) * 100

resumen_nulos = pd.DataFrame({
    'Columna': df.columns,
    'Nulos': nulos.values,
    'Porcentaje (%)': porcentaje_nulos.values.round(2)
})

print(resumen_nulos.to_string(index=False))
print("\n" + "=" * 60)
print(f"Total de valores nulos en todo el dataset: {df.isnull().sum().sum():,}")

Valores Nulos por Columna:
               Columna  Nulos  Porcentaje (%)
           Feedback_ID      0            0.00
        Transaccion_ID      0            0.00
       Rating_Producto      0            0.00
      Rating_Logistica      0            0.00
      Comentario_Texto    657           14.60
      Recomienda_Marca   1119           24.87
Ticket_Soporte_Abierto      0            0.00
          Edad_Cliente      0            0.00
      Satisfaccion_NPS      0            0.00

Total de valores nulos en todo el dataset: 1,776


In [21]:
# Recuento de valores únicos en Comentario_Texto
print("Recuento de Valores Únicos en 'Comentario_Texto':")
print("=" * 60)

comentarios_conteo = df['Comentario_Texto'].value_counts()

print(f"Total de valores únicos: {df['Comentario_Texto'].nunique()}")
print(f"Total de registros: {len(df)}")
print("\n" + "-" * 60)
print("\nDistribución de comentarios:")
print(comentarios_conteo)
print("\n" + "=" * 60)
print(f"\nLos 5 comentarios más frecuentes:")
print(comentarios_conteo.head())

Recuento de Valores Únicos en 'Comentario_Texto':
Total de valores únicos: 6
Total de registros: 4500

------------------------------------------------------------

Distribución de comentarios:
Comentario_Texto
Excelente       677
Lento           668
Dañado          647
---             631
No volvería     624
Precio justo    596
Name: count, dtype: int64


Los 5 comentarios más frecuentes:
Comentario_Texto
Excelente      677
Lento          668
Dañado         647
---            631
No volvería    624
Name: count, dtype: int64


In [22]:
# Conteo de valores únicos en las columnas de Rating
print("Conteo de Valores Únicos - Rating_Producto:")
print("=" * 60)
rating_producto_conteo = df['Rating_Producto'].value_counts().sort_index()
print(rating_producto_conteo)
print(f"\nTotal de valores únicos: {df['Rating_Producto'].nunique()}")

print("\n" + "=" * 60)
print("Conteo de Valores Únicos - Rating_Logistica:")
print("=" * 60)
rating_logistica_conteo = df['Rating_Logistica'].value_counts().sort_index()
print(rating_logistica_conteo)
print(f"\nTotal de valores únicos: {df['Rating_Logistica'].nunique()}")

Conteo de Valores Únicos - Rating_Producto:
Rating_Producto
1     922
2     876
3     903
4     837
5     932
99     30
Name: count, dtype: int64

Total de valores únicos: 6

Conteo de Valores Únicos - Rating_Logistica:
Rating_Logistica
1    919
2    852
3    922
4    901
5    906
Name: count, dtype: int64

Total de valores únicos: 5


In [24]:
# Conteo de valores únicos en las columnas restantes
columnas_restantes = ['Recomienda_Marca', 'Ticket_Soporte_Abierto', 'Edad_Cliente', 'Satisfaccion_NPS']

for columna in columnas_restantes:
    print(f"Conteo de Valores Únicos - {columna}:")
    print("=" * 60)
    conteo = df[columna].value_counts().sort_index()
    print(conteo)
    print(f"\nTotal de valores únicos: {df[columna].nunique()}")
    print(f"Total de registros: {len(df)}")
    print("\n" + "-" * 60 + "\n")

Conteo de Valores Únicos - Recomienda_Marca:
Recomienda_Marca
Maybe    1077
NO       1142
SI       1162
Name: count, dtype: int64

Total de valores únicos: 3
Total de registros: 4500

------------------------------------------------------------

Conteo de Valores Únicos - Ticket_Soporte_Abierto:
Ticket_Soporte_Abierto
0     1117
1     1140
No    1085
Sí    1158
Name: count, dtype: int64

Total de valores únicos: 4
Total de registros: 4500

------------------------------------------------------------

Conteo de Valores Únicos - Edad_Cliente:
Edad_Cliente
18     74
19     72
20     66
21     57
22     64
23     71
24     79
25     68
26     94
27     78
28     63
29     74
30     56
31     72
32     59
33     73
34     44
35     79
36     57
37     66
38     65
39     86
40     73
41     61
42     52
43     75
44     59
45     71
46     74
47     60
48     73
49     65
50     73
51     64
52     62
53     75
54     70
55     64
56     75
57     75
58     62
59     67
60     58
61     79


In [27]:
# Transformar columna Ticket_Soporte_Abierto
print("Transformación de Ticket_Soporte_Abierto:")
print("=" * 60)
print("Valores únicos antes de la transformación:")
print(df['Ticket_Soporte_Abierto'].value_counts())

# Reemplazar valores (incluyendo posibles valores '0' y '1' como strings)
df['Ticket_Soporte_Abierto'] = df['Ticket_Soporte_Abierto'].replace({
    'Sí': 1, 
    'No': 0,
    '1': 1,
    '0': 0
})

# Convertir a tipo numérico para asegurar consistencia
df['Ticket_Soporte_Abierto'] = pd.to_numeric(df['Ticket_Soporte_Abierto'], errors='coerce')

print("\n" + "-" * 60)
print("Valores únicos después de la transformación:")
print(df['Ticket_Soporte_Abierto'].value_counts().sort_index())

print("\n" + "=" * 60)
print("✓ Transformación completada")

Transformación de Ticket_Soporte_Abierto:
Valores únicos antes de la transformación:
Ticket_Soporte_Abierto
1    1158
1    1140
0    1117
0    1085
Name: count, dtype: int64

------------------------------------------------------------
Valores únicos después de la transformación:
Ticket_Soporte_Abierto
0    2202
1    2298
Name: count, dtype: int64

✓ Transformación completada


/tmp/ipykernel_11771/559631835.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Ticket_Soporte_Abierto'] = df['Ticket_Soporte_Abierto'].replace({


In [28]:
# Descripción estadística de todas las columnas
print("DESCRIPCIÓN ESTADÍSTICA DE COLUMNAS NUMÉRICAS:")
print("=" * 80)
print(df.describe())

print("\n" + "=" * 80)
print("DESCRIPCIÓN ESTADÍSTICA DE COLUMNAS NO NUMÉRICAS:")
print("=" * 80)
print(df.describe(include=['object']))

print("\n" + "=" * 80)
print("RESUMEN COMPLETO - TODAS LAS COLUMNAS:")
print("=" * 80)
print(df.describe(include='all'))

DESCRIPCIÓN ESTADÍSTICA DE COLUMNAS NUMÉRICAS:
       Rating_Producto  Rating_Logistica  Ticket_Soporte_Abierto  \
count      4500.000000       4500.000000             4500.000000   
mean          3.635778          3.005111                0.510667   
std           7.942196          1.418520                0.499942   
min           1.000000          1.000000                0.000000   
25%           2.000000          2.000000                0.000000   
50%           3.000000          3.000000                1.000000   
75%           4.000000          4.000000                1.000000   
max          99.000000          5.000000                1.000000   

       Edad_Cliente  Satisfaccion_NPS  
count   4500.000000       4500.000000  
mean      51.172889          0.303089  
std       21.830784         57.184315  
min       18.000000        -99.800000  
25%       34.000000        -49.200000  
50%       50.000000          1.100000  
75%       67.000000         49.525000  
max      195.000000 